# Consulta o banco de dados Oracle

In [ ]:
import pandas as pd
import oracledb
# Ajustar a configuração do Pandas para mostrar todas as linhas
#pd.set_option('display.max_rows', None)

# Instalar pip install SQLAlchemy
#from sqlalchemy import create_engine

# Minhas credenciais
credencial_banco = {
    'dsn': 'oracle.fiap.com.br:1521/orcl',
    'usuario': 'rm553472',
    'senha': '100593'
}

# Função para conectar ao banco de dados
def conectar(usuario, senha, dsn):
    try:
        conexao = oracledb.connect(user=usuario, password=senha, dsn=dsn, mode=oracledb.DEFAULT_AUTH)
        print("Conexão com o banco de dados Oracle estabelecida com sucesso.")
        return conexao
    except oracledb.DatabaseError as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

conexao = conectar(usuario='rm553472',senha='100593', dsn='oracle.fiap.com.br:1521/orcl')

# Consultar dados da tabela de feedback
query_feedback = """
SELECT *
FROM feedback
"""

# Carregar dados em um DataFrame do Pandas
df_feedback = pd.read_sql(query_feedback, conexao)

# Fechar a conexão com o banco de dados
conexao.close()

In [ ]:
df_feedback

# Tratar os dados da tabela Feedback

In [ ]:
# Selecionar as colunas necessárias para o treinamento do modelo de classificação de feedback
df_feedback_model = df_feedback[['NOTA', 'COMENTARIO']]

# Exibir as primeiras linhas do DataFrame preparado para o modelo
df_feedback_model.head()

# Salvar o DataFrame preparado em um arquivo CSV para uso posterior pelo professor
df_feedback_model.to_csv('feedback_model_data.csv', index=False)


In [ ]:
df_feedback_model.info()

In [ ]:
df_feedback_model['NOTA'] = df_feedback_model['NOTA'].astype(int)

In [ ]:
df_feedback_model

# Treinar o modelo para classificar os Feedbacks

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Carregar os dados do arquivo CSV
#df_feedback_model = pd.read_csv('feedback_model_data.csv')

# Separar os dados em recursos (X) e rótulos (y)
X = df_feedback_model['COMENTARIO']
y = df_feedback_model['NOTA']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização dos comentários usando TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar o modelo de classificação usando RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train_tfidf, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test_tfidf)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred))

# Salvar o modelo treinado e o vetor de palavras para uso posterior
joblib.dump(model, 'feedback_classification_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
import joblib

# Carregar os dados do arquivo CSV
#df_feedback_model = pd.read_csv('feedback_model_data.csv')

# Converter a coluna 'avaliacao' para o tipo inteiro
df_feedback_model['NOTA'] = df_feedback_model['NOTA'].astype(int)

# Balancear o conjunto de dados
df_majority = df_feedback_model[df_feedback_model['NOTA'] == 4]
df_minority = df_feedback_model[df_feedback_model['NOTA'] == 5]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results

df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Separar os dados em recursos (X) e rótulos (y)
X = df_balanced['COMENTARIO']
y = df_balanced['NOTA']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização dos comentários usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar o modelo de classificação usando RandomForestClassifier com ajuste de hiperparâmetros
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train_tfidf, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test_tfidf)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred))

# Salvar o modelo treinado e o vetor de palavras para uso posterior
joblib.dump(model, 'feedback_classification_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


## Sugerir consultas

1. Coletar Dados do Cliente e das Clínicas:

1.1. Informações do cliente: dia da semana preferido, horário, turno, endereço.

1.2. Informações das clínicas: endereço, especialidades, notas de feedback e custo.

2. Calcular a distância entre o endereço do cliente e das clínicas.

3. Normalizar as avaliações e outras características relevantes da pesquisa de satisfação. Neste caso, posso usar o modelo treinado por API.

4. Utilizar um algoritmo de aprendizado de máquina para treinar o modelo com os dados processados.

5. Criar uma API que utilize o modelo treinado para sugerir consultas com base nas preferências do cliente. Vamos usar está API no front Mobile e Site.

In [ ]:
Novo treinamento

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Carregar os dados
data = pd.read_csv('feedback_model_data.csv')

# Pré-processamento
X = data['COMENTARIO']
y = data['NOTA']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar o modelo
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

# Exportar o modelo treinado para um arquivo
joblib.dump(model, 'modelo_treinado_segundo.pkl')

# Analisar sentimento

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import joblib
import scipy

# Carregar os dados
data = pd.read_csv('feedback_model_data.csv')

# Função para análise de sentimento
def sentiment_score(comment):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(comment)
    return score['compound']

# Adicionar coluna de sentimento
data['SENTIMENTO'] = data['COMENTARIO'].apply(sentiment_score)

# Pré-processamento
X = data[['COMENTARIO', 'SENTIMENTO']]
y = data['NOTA']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['COMENTARIO'])
X_test_tfidf = vectorizer.transform(X_test['COMENTARIO'])

# Adicionar a coluna de sentimento
import scipy
X_train_tfidf = scipy.sparse.hstack((X_train_tfidf, X_train[['SENTIMENTO']].values))
X_test_tfidf = scipy.sparse.hstack((X_test_tfidf, X_test[['SENTIMENTO']].values))

# Treinar o modelo
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

# Exportar o modelo treinado e o vetorizador para arquivos
joblib.dump(model, 'modelo_treinado.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')
